In [118]:
import sympy as sym
import numpy as np
import numba as nb

from IPython.display import display, Math

from einsteinpy.symbolic import BaseRelativityTensor, ChristoffelSymbols
from einsteinpy.symbolic.tensor import Tensor

from fpdf import FPDF
import pylatex as pltx

sym.init_printing()

In [27]:
r, M, a, Q, theta, T, gamma = sym.symbols('r, M, a, Q, theta, T, gamma')
rho, Delta = sym.symbols('rho, Delta')

In [82]:
def connCoeff(alpha, mu, nu):
    h = r**2 + a**2
    rho_sub = r**2 + a**2 * sym.cos(theta)**2
    Delta_sub = r**2 - 2*M*r + a**2 + Q**2

    g_mu_nu = sym.diag(
        Delta/rho**2,
        -rho**2/Delta,
        -rho**2,
        -h**2 * sym.sin(theta**2) / rho**2
    )
    g_mu_nu_inv = g_mu_nu.inv()

    diff_arr = [T, r, theta, gamma]
    gmn_l = g_mu_nu.subs([(rho**2, rho_sub), (Delta, Delta_sub)]).tolist()
    gmn_i_l = g_mu_nu_inv.subs([(rho**2, rho_sub), (Delta, Delta_sub)]).tolist()

    conn = gmn_i_l[alpha][alpha] * (
        sym.diff(gmn_l[nu][alpha], diff_arr[mu]) + 
        sym.diff(gmn_l[alpha][mu], diff_arr[nu]) -
        sym.diff(gmn_l[mu][nu], diff_arr[alpha])
    )

    conn *= sym.Rational(1,2)
    conn = conn.subs([(rho_sub, rho**2), (Delta_sub, Delta)])

    return conn

In [83]:
connCoeff(2, 2, 1).simplify()

In [106]:
def getConnCoeffMatrix():
    conn_coeff_matrix = np.zeros(shape=(4,4,4)).tolist()
    for i in range(4):
        for j in range(4):
            for k in range(4):
                conn_coeff_matrix[i][j][k] = connCoeff(i, j, k).simplify()

    return conn_coeff_matrix

In [107]:
conn_coeff_matrix = getConnCoeffMatrix()

In [156]:
for i in range(4):
    display(Math(sym.latex(sym.symbols('alpha')) + f"= {str(i)}"))
    for j in range(4):
        text = ""
        for k in range(4):
            text += f'{sym.latex(conn_coeff_matrix[i][j][k])},'
        display(Math(f'{sym.latex(sym.symbols("mu"))} = {j} =>\ {sym.latex(sym.symbols("nu"))} =' + text[:-1]))
            
        print("\n")

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>